### Part #1: Import Libraries/Files & Setup Main Dataset

In [182]:
# Dependencies; import Pandas library & setup pathlib to load csv files
import pandas as pd
from pathlib import Path

# Load csv dataset files
load_school_data = Path("Resources/schools_complete.csv")
load_student_data = Path("Resources/students_complete.csv")

# Read school & student files and store datasets into separate Pandas DataFrames
school_df = pd.read_csv(load_school_data)
student_df = pd.read_csv(load_student_data)

# Merge both datasets into one;  
complete_school_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

# remove 'Student ID' & 'School ID'; not necessary for calculations and dataset already has pandas index
del complete_school_df["Student ID"]
del complete_school_df["School ID"]

# Display first 5 rows of the complete DataFrame
complete_school_df.head()

,student_name,gender,year,school_name,reading_score,maths_score,type,size,budget
0,Paul Bradley,M,9,Huang High School,96,94,Government,2917,1910635
1,Victor Smith,M,12,Huang High School,90,43,Government,2917,1910635
2,Kevin Rodriguez,M,12,Huang High School,41,76,Government,2917,1910635
3,Richard Scott,M,12,Huang High School,89,86,Government,2917,1910635
4,Bonnie Ray,F,9,Huang High School,87,69,Government,2917,1910635


### Part #2: Local Government Area (LGA) Summary Table

In [183]:
# Total schools; count unique (distinct) values from 'school_name' column
total_schools = complete_school_df["school_name"].nunique()

# Total Students; length of the complete dataset
total_students = len(complete_school_df)

# Total Budget; sum of all unique values from 'budget' column; ignore duplicates
total_budget = complete_school_df["budget"].unique().sum()

# Average Math Score; mean of all values from 'maths_score' column
average_math = complete_school_df["maths_score"].mean()

# Average Reading Score; mean of all values from 'reading_score' column
average_reading = complete_school_df["reading_score"].mean()

# % students w/ passing maths score; dataset length (w/ rows of math score >= 50 only) divided by total students
pass_math_df = complete_school_df.loc[complete_school_df["maths_score"] >= 50]
percent_pass_math = len(pass_math_df) / total_students

# % students w/ passing reading score; dataset length (w/ rows of reading score >= 50 only) divided by total students
pass_reading_df = complete_school_df.loc[complete_school_df["reading_score"] >= 50]
percent_pass_reading = len(pass_reading_df) / total_students

# % students w/ overall passing score; dataset length (w/ rows where math & reading scores are both >= 50 only) / total students
pass_overall_df = complete_school_df.loc[(complete_school_df["maths_score"] >= 50) &
                                         (complete_school_df["reading_score"] >= 50)]
percent_pass_overall = len(pass_overall_df) / total_students

# Store calculated summary values in new DataFrame; pass through dictionary containing the column names & summary values
area_summary = pd.DataFrame({
                                "Total Schools": [total_schools],
                                "Total Students": [total_students],
                                "Total Budget": [total_budget],
                                "Average Maths Score": [average_math],
                                "Average Reading Score": [average_reading],
                                "% Passing Maths": [percent_pass_math],
                                "% Passing Reading": [percent_pass_reading],
                                "% Overall Passing": [percent_pass_overall]})

# Format relevant data in Area Summary DataFrame for cleaner display
area_summary["Total Students"] = area_summary["Total Students"].map("{:,.0f}".format)       # Format in thousands to 0 d.p.
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)          # Format in thousands to 2 d.p.
area_summary["% Passing Maths"] = area_summary["% Passing Maths"].map("{:,.6%}".format)     # % Format to 6 d.p.
area_summary["% Passing Reading"] = area_summary["% Passing Reading"].map("{:,.6%}".format) # % Format to 6 d.p.
area_summary["% Overall Passing"] = area_summary["% Overall Passing"].map("{:,.6%}".format) # % Format to 6 d.p.

# Display 'area_summary' DataFrame
area_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632%,84.426857%,72.808272%


### Part #3: Per School Summary Overview Table

In [184]:
# Create new list to store all distinct school names from the complete dataset; to be iterated using for loop
school_list = complete_school_df["school_name"].unique()

#Create empty DataFrame to store key metrics of each school; columns and their names only defined
per_school_summary = pd.DataFrame(columns = ["School Name",
                                             "School Type",
                                             "Total Students",
                                             "Total School Budget",
                                             "Per Student Budget",
                                             "Average Maths Score",
                                             "Average Reading Score",
                                             "% Passing Maths",
                                             "% Passing Reading",
                                             "% Overall Passing"])


# For every School Name in the school list...
# In temp DataFrame, store specific rows & columns from the complete DataFrame where school name is current name from list
# During iteration, use temp DataFrame to calculate summary calculations for current school and pass data to main DataFrame
for i in range(len(school_list)):
    temp_df = complete_school_df.loc[complete_school_df["school_name"] == school_list[i]]
    
    # Get current school name; use iloc to get school name from first row in temp DataFrame
    temp_school_name = temp_df["school_name"].iloc[0]
    
    # Get current school type; use iloc to get school type from first row in temp DataFrame
    temp_school_type = temp_df["type"].iloc[0]
    
    # Get total of students for current school; length of temp DataFrame
    temp_student_total = len(temp_df)
    
    # Get total of budget for current school; use unique value from temp DataFrame
    temp_budget_total = temp_df["budget"].unique().sum()
    
    # Get budget per student for current school; total budget divided by total students
    temp_student_budget = temp_budget_total / temp_student_total
    
    # Get average maths score for current school; mean of all values from 'maths_score' from temp DataFrame
    temp_average_math = temp_df["maths_score"].mean()
    
    # Get average reading score for current school; mean of all values from 'reading_score' from temp DataFrame
    temp_average_reading = temp_df["reading_score"].mean()
    
    # Get % of students w/ passing maths score for current school; use new temp DataFrame data where maths score >= 50 
    temp_pass_math_df = temp_df.loc[temp_df["maths_score"] >= 50]
    temp_percent_pass_math = len(temp_pass_math_df) / temp_student_total
    
    # Get % of students w/ passing reading score for current school; use new temp DataFrame data where reading score >= 50 
    temp_pass_reading_df = temp_df.loc[temp_df["reading_score"] >= 50]
    temp_percent_pass_reading = len(temp_pass_reading_df) / temp_student_total
    
    # Get % of students w/ overall passing score for current school; use new temp DataFrame where reading & math scores >= 50
    temp_pass_overall_df = temp_df.loc[(temp_df["maths_score"] >= 50) & (temp_df["reading_score"] >= 50)]
    temp_percent_pass_overall = len(temp_pass_overall_df) / temp_student_total
    
    # Insert all summary data for current school in a new row to the Per School Summary DataFrame
    per_school_summary.loc[i] = [temp_school_name,
                                 temp_school_type,
                                 temp_student_total,
                                 temp_budget_total,
                                 temp_student_budget,
                                 temp_average_math,
                                 temp_average_reading,
                                 temp_percent_pass_math,
                                 temp_percent_pass_reading,
                                 temp_percent_pass_overall]

    
# Format relevant data in Per School Summary DataFrame for cleaner display
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format) # Format in 1000s (2 d.p.)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)          # Format in thousands to 2 d.p.
per_school_summary["% Passing Maths"] = per_school_summary["% Passing Maths"].map("{:,.6%}".format)     # % Format to 6 d.p.
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].map("{:,.6%}".format) # % Format to 6 d.p.
per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].map("{:,.6%}".format) # % Format to 6 d.p.

# In 'per_school_summary' DataFrame, set "School Name" column as index; also remove index column name
per_school_summary.set_index("School Name", inplace=True)
per_school_summary.index.name = None

# Display 'per_school_summary' DataFrame; sort by index i.e. alphabetically
per_school_summary.sort_index()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871%,87.379421%,80.084405%
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377%,89.074273%,80.785791%
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798%,82.807731%,67.650051%
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846%,82.219788%,67.469880%
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534%,88.487738%,81.335150%
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299%,81.877023%,66.364617%
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742%,88.524590%,78.922717%
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521%,81.453548%,66.712376%
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592%,81.978576%,67.191766%
Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,91.683992%,86.590437%,79.209979%


### Part #4: Top Performing Schools (By % Overall Passing)

In [185]:
# Store 'per_school_summary' to new DataFrame where sorted by '% Overall Passing' in descending order
top_schools = per_school_summary.sort_values(by="% Overall Passing", ascending = False)

# Display first five rows from 'top_schools' DataFrame i.e. show Top 5 Performing Schools
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534%,88.487738%,81.335150%
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377%,89.074273%,80.785791%
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871%,87.379421%,80.084405%
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778%,86.666667%,79.722222%
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699%,87.396849%,79.419855%


### Part #5: Bottom Performing Schools (By % Overall Passing)

In [186]:
# Store 'per_school_summary' to new DataFrame where sorted by '% Overall Passing' in ascending order
bottom_schools = per_school_summary.sort_values(by="% Overall Passing", ascending = True)

# Display first five rows from 'bottom_schools' DataFrame i.e. show Bottom 5 Performing Schools
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299%,81.877023%,66.364617%
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521%,81.453548%,66.712376%
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592%,81.978576%,67.191766%
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808%,81.296540%,67.455103%
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846%,82.219788%,67.469880%


### Part #6: Maths Scores by Year

In [187]:
# Create new list to store all distinct school years from the complete dataset; to be iterated via nester for loop
year_list = complete_school_df["year"].unique()
year_list.sort()

#Create empty DataFrame to store average math score for every school year; columns and their names only defined
maths_scores_by_year = pd.DataFrame(columns = ["School Name",
                                               "Year 9",
                                               "Year 10",
                                               "Year 11",
                                               "Year 12"])

# For every School Name in the school list (declared earlier)...
for i in range(len(school_list)):
    
    #create new list and current school name to it; visualised as new row for the 'maths_scores_by_year' DataFrame
    new_row_math = [school_list[i]]
    
    # For every school year in the year list...
    # In new temp DataFrame, store specific data where school name is current name & school year is current year
    for j in range(len(year_list)):
        temp_year_math_df = complete_school_df.loc[(complete_school_df["school_name"] == school_list[i]) &
                                                   (complete_school_df["year"] == year_list[j])]
        
        # Get the mean of maths score
        temp_year_average_math = temp_year_math_df["maths_score"].mean()
        
        # Append new row (list) with the calculated average math score for current year; iterate inner loop
        new_row_math.append(temp_year_average_math)
    
    # Append 'maths_scores_by_year' with new row (list); iterate outer loop
    maths_scores_by_year.loc[i] =  new_row_math
    

# In 'maths_scores_by_year' DataFrame, set "School Name" column as index; also remove index column name
maths_scores_by_year.set_index("School Name", inplace=True)
maths_scores_by_year.index.name = None    

# Display 'maths_scores_by_year' DataFrame; sort by index i.e. alphabetically
maths_scores_by_year.sort_index()

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


### Part #7: Reading Scores by Year

In [188]:
#Create empty DataFrame to store average reading score for every school year; columns and their names only defined
reading_scores_by_year = pd.DataFrame(columns = ["School Name",
                                                 "Year 9",
                                                 "Year 10",
                                                 "Year 11",
                                                 "Year 12"])

# For every School Name in the school list (declared earlier)...
for i in range(len(school_list)):
    
    #create new list and current school name to it; visualised as new row for the 'reading_scores_by_year' DataFrame
    new_row_reading = [school_list[i]]
    
    # For every school year in the year list (declared earlier)...
    # In new temp DataFrame, store specific data where school name is current name & school year is current year
    for j in range(len(year_list)):
        temp_year_reading_df = complete_school_df.loc[(complete_school_df["school_name"] == school_list[i]) &
                                                   (complete_school_df["year"] == year_list[j])]
        
        # Get the mean of maths score
        temp_year_average_reading = temp_year_reading_df["reading_score"].mean()
        
        # Append new row (list) with the calculated average math score for current year; iterate inner loop
        new_row_reading.append(temp_year_average_reading)
    
    # Append 'maths_scores_by_year' with new row (list); iterate outer loop
    reading_scores_by_year.loc[i] = new_row_reading
    

# In 'reading_scores_by_year' DataFrame, set "School Name" column as index; also remove index column name
reading_scores_by_year.set_index("School Name", inplace=True)
reading_scores_by_year.index.name = None    

# Display 'maths_scores_by_year' DataFrame; sort by index i.e. alphabetically
reading_scores_by_year.sort_index()

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


### Part #8: Scores by School Spending

In [191]:
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].str.replace('$', '')
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].astype(float)

per_school_summary["% Passing Maths"] = per_school_summary["% Passing Maths"].str.replace('%', '')
per_school_summary["% Passing Maths"] = per_school_summary["% Passing Maths"].astype(float)

per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].str.replace('%', '')
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].astype(float)

per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].str.replace('%', '')
per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].astype(float)

per_school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Huang High School,Government,2917,"$1,910,635.00",655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Figueroa High School,Government,2949,"$1,884,411.00",639.0,68.698542,69.077993,81.654798,82.807731,67.650051
Shelton High School,Independent,1761,"$1,056,600.00",600.0,72.034072,70.257808,91.538898,86.712095,78.875639
Hernandez High School,Government,4635,"$3,022,020.00",652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Griffin High School,Independent,1468,"$917,500.00",625.0,71.788147,71.245232,91.212534,88.487738,81.335150


In [206]:
# Create list to store spending range ('Per Student Budget'); No. intervals (bins) to divide data from 'per_school_summary'
spending_bins = [0, 585, 630, 645, 680]

# Create list to store the labels for each bin
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

per_school_summary["Spending Ranges (Per Student)"] = pd.cut(per_school_summary["Per Student Budget"],
                                                             spending_bins, labels = spending_labels,
                                                             include_lowest = True)

spending_average_math = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_average_reading = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_percent_pass_math = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()
spending_percent_pass_reading = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
spending_percent_pass_overall = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

#spending_summary = pd.merge(spending_average_math, spending_average_reading, on = ["Spending Ranges (Per Student)"])
spending_summary = pd.concat([spending_average_math,
                              spending_average_reading,
                              spending_percent_pass_math,
                              spending_percent_pass_reading,
                              spending_percent_pass_overall],
                              axis = 1)

spending_summary.round(2)

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


### Part #9: Scores by School Size

In [207]:
# Create list to store school size ('Total Students'); No. intervals (bins) to divide data from 'per_school_summary'
size_bins = [0, 1000, 2000, 5000]

# Create list to store the labels for each bin
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"],
                                                             size_bins, labels = size_labels,
                                                             include_lowest = True)

size_average_math = per_school_summary.groupby(["School Size"])["Average Maths Score"].mean()
size_average_reading = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_percent_pass_math = per_school_summary.groupby(["School Size"])["% Passing Maths"].mean()
size_percent_pass_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_percent_pass_overall = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

#spending_summary = pd.merge(spending_average_math, spending_average_reading, on = ["Spending Ranges (Per Student)"])
size_summary = pd.concat([size_average_math,
                          size_average_reading,
                          size_percent_pass_math,
                          size_percent_pass_reading,
                          size_percent_pass_overall],
                          axis = 1)

size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714148,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


### Part #10: Scores by School Type

In [208]:
type_average_math = per_school_summary.groupby(["School Type"])["Average Maths Score"].mean()
type_average_reading = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
type_percent_pass_math = per_school_summary.groupby(["School Type"])["% Passing Maths"].mean()
type_percent_pass_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
type_percent_pass_overall = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

#spending_summary = pd.merge(spending_average_math, spending_average_reading, on = ["Spending Ranges (Per Student)"])
type_summary = pd.concat([type_average_math,
                          type_average_reading,
                          type_percent_pass_math,
                          type_percent_pass_reading,
                          type_percent_pass_overall],
                          axis = 1)

type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204042,86.247789,76.973340
